In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
dataset_path = '/content/drive/MyDrive/animal_data'

In [4]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix

In [5]:
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2
)

In [6]:
train_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=(128, 128),
    batch_size=16,
    class_mode='categorical',
    subset='training'
)

Found 3679 images belonging to 4 classes.


In [7]:
validation_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=(128, 128),
    batch_size=16,
    class_mode='categorical',
    subset='validation'
)

Found 916 images belonging to 4 classes.


In [8]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(4, activation='softmax')
])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [9]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [10]:
epochs = 10
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=epochs,
    verbose=1
)

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


229/229 ━━━━━━━━━━━━━━━━━━━━ 1173s 5s/step - accuracy: 0.2808 - loss: 1.3797 - val_accuracy: 0.4737 - val_loss: 1.1545
Epoch 2/10
  1/229 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - accuracy: 0.2500 - loss: 1.3344

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


229/229 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.2500 - loss: 1.3344 - val_accuracy: 0.5000 - val_loss: 1.0433
Epoch 3/10
229/229 ━━━━━━━━━━━━━━━━━━━━ 91s 182ms/step - accuracy: 0.4726 - loss: 1.1763 - val_accuracy: 0.5636 - val_loss: 1.0066
Epoch 4/10
229/229 ━━━━━━━━━━━━━━━━━━━━ 0s 125us/step - accuracy: 0.4375 - loss: 1.0344 - val_accuracy: 0.2500 - val_loss: 1.1701
Epoch 5/10
229/229 ━━━━━━━━━━━━━━━━━━━━ 42s 181ms/step - accuracy: 0.5194 - loss: 1.0843 - val_accuracy: 0.5866 - val_loss: 0.9462
Epoch 6/10
229/229 ━━━━━━━━━━━━━━━━━━━━ 0s 138us/step - accuracy: 0.6250 - loss: 0.9939 - val_accuracy: 0.2500 - val_loss: 1.0959
Epoch 7/10
229/229 ━━━━━━━━━━━━━━━━━━━━ 81s 178ms/step - accuracy: 0.5539 - loss: 1.0200 - val_accuracy: 0.5844 - val_loss: 0.9292
Epoch 8/10
229/229 ━━━━━━━━━━━━━━━━━━━━ 0s 141us/step - accuracy: 0.4375 - loss: 1.0709 - val_accuracy: 0.5000 - val_loss: 0.7974
Epoch 9/10
229/229 ━━━━━━━━━━━━━━━━━━━━ 82s 176ms/step - accuracy: 0.5689 - loss: 0.9762 - val_accur

In [12]:
from tensorflow.keras.models import save_model

model_save_path = '/content/drive/MyDrive/animal_model.keras'

model.save(model_save_path)

print(f"Model saved to {model_save_path}")

Model saved to /content/drive/MyDrive/animal_model.keras


In [13]:
validation_generator.reset()
preds = model.predict(validation_generator, steps=validation_generator.samples // validation_generator.batch_size + 1)
y_pred = np.argmax(preds, axis=1)

print('Classification Report')
print(classification_report(validation_generator.classes, y_pred, target_names=validation_generator.class_indices.keys()))

print('Confusion Matrix')
print(confusion_matrix(validation_generator.classes, y_pred))

58/58 ━━━━━━━━━━━━━━━━━━━━ 12s 195ms/step
Classification Report
              precision    recall  f1-score   support

         cat       0.27      0.30      0.29       237
         dog       0.23      0.10      0.14       237
    elephant       0.25      0.31      0.27       205
       tiger       0.27      0.34      0.30       237

    accuracy                           0.26       916
   macro avg       0.26      0.26      0.25       916
weighted avg       0.26      0.26      0.25       916

Confusion Matrix
[[72 28 78 59]
 [61 24 60 92]
 [59 23 63 60]
 [74 29 54 80]]
